<a href="https://colab.research.google.com/github/krishnamanc/PytorchPractice/blob/main/FirstNetworkOwn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
trainloader = data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=0)
testloader = data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
trainset.data.shape

torch.Size([60000, 28, 28])

In [ ]:
testset.data.shape

torch.Size([10000, 28, 28])

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)


  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
model = Model()

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
epochs = 5

In [ ]:
for epoch in range(epochs):
  for i, (inputs, labels) in enumerate(trainloader):
    # train time

    model.train()

    inputs = inputs.view(-1, 784)

    outputs = model(inputs)

    loss = loss_fn(outputs, labels)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/5, Loss: 0.015517921186983585
Epoch 2/5, Loss: 7.711553189437836e-05
Epoch 3/5, Loss: 0.0008682420011609793
Epoch 4/5, Loss: 5.773723842139589e-06
Epoch 5/5, Loss: 3.352759847530251e-08


In [ ]:
correct, total = 0,0
predictions = []

for i, data in enumerate(testloader):
  inputs, labels = data
  inputs = inputs.view(-1, 784)

  with torch.no_grad():
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    predictions.append(outputs)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")

Accuracy of the network on the 10000 test images: 98.03%


In [ ]:
torch.save(model.state_dict(), 'model.pth')